.. _tutorial_coverage:

# Execution and Coverage Reporting

The core component of the notebook execution API is the :py:class:`~pytest_notebook.execution.ExecuteCoveragePreprocessor` class,
which is a subclass of :py:class:`nbconvert.preprocessors.ExecutePreprocessor`,
that can additionally create code [coverage](https://coverage.readthedocs.io) analytics.

This class is called by :py:func:`~pytest_notebook.execution.execute_notebook`,
which returns an :py:class:`~pytest_notebook.execution.ExecuteResult` object.

In [1]:
import ruamel.yaml as yaml
from pytest_notebook.execution import execute_notebook
from pytest_notebook.notebook import create_notebook, create_cell, dump_notebook

In [2]:
notebook = create_notebook()
notebook.cells = [
    create_cell("""
from pytest_notebook import __version__
from pytest_notebook.notebook import create_notebook
print(__version__)
"""
    )
]

In [3]:
result = execute_notebook(notebook, with_coverage=True)
result.notebook

{'nbformat': 4,
 'nbformat_minor': 2,
 'metadata': {'language_info': {'name': 'python',
   'version': '3.6.7',
   'mimetype': 'text/x-python',
   'codemirror_mode': {'name': 'ipython', 'version': 3},
   'pygments_lexer': 'ipython3',
   'nbconvert_exporter': 'python',
   'file_extension': '.py'}},
 'cells': [{'cell_type': 'code',
   'metadata': {},
   'execution_count': 1,
   'source': '\nfrom pytest_notebook import __version__\nfrom pytest_notebook.notebook import create_notebook\nprint(__version__)\n',
   'outputs': [{'output_type': 'stream',
     'name': 'stdout',
     'text': '0.6.0\n'}]}]}

In [26]:
result.coverage_data()

<CoverageData lines={5} arcs=None tracers={0} runs=[0]>

In [4]:
print(yaml.dump(result.coverage_dict))

lines:
  /Users/cjs14/GitHub/pytest-notebook/pytest_notebook/__init__.py: [4, 5]
  /Users/cjs14/GitHub/pytest-notebook/pytest_notebook/diffing.py: [1, 2, 3, 4, 5,
    7, 8, 9, 10, 11, 12, 24, 25, 66, 67, 98, 101, 116, 117, 162, 163]
  /Users/cjs14/GitHub/pytest-notebook/pytest_notebook/notebook.py: [1, 2, 3, 4, 5,
    6, 8, 9, 10, 11, 12, 13, 15, 16, 17, 19, 21, 25, 26, 49, 50, 80, 81, 108, 109,
    112, 142, 151, 173, 174, 176, 178, 179, 180, 181, 184, 191, 192, 195, 202, 203,
    204, 205, 207, 208, 209, 210, 214, 245, 246, 252, 253, 260, 267, 275, 291, 292,
    293, 295, 308]
  /Users/cjs14/GitHub/pytest-notebook/pytest_notebook/resources/__init__.py: [1]
  /Users/cjs14/GitHub/pytest-notebook/pytest_notebook/utils.py: [1, 2, 4, 5, 6, 9,
    29, 52, 86, 99, 136, 138, 142, 144, 100, 102, 106, 108, 112, 32, 35, 40, 41, 44,
    45, 49, 114, 115, 116, 120, 122, 123, 124, 125, 126, 128, 134, 149, 151, 87, 89,
    97, 156]



The coverage can be limited to particular files or modules, by setting `cov_source`.

In [28]:
result = execute_notebook(
    notebook, with_coverage=True, cov_source=['pytest_notebook.notebook'])
print(yaml.dump(result.coverage_dict))

lines:
  /Users/cjs14/GitHub/pytest-notebook/pytest_notebook/notebook.py: [1, 2, 3, 4, 5,
    6, 8, 9, 10, 11, 12, 13, 15, 16, 17, 19, 21, 25, 26, 49, 50, 80, 81, 108, 109,
    112, 142, 151, 173, 174, 176, 178, 179, 180, 181, 184, 191, 192, 195, 202, 203,
    204, 205, 207, 208, 209, 210, 214, 245, 246, 252, 253, 260, 267, 275, 291, 292,
    293, 295, 308]



## Integration with pytest-cov

If the [pytest-cov](https://pytest-cov.readthedocs.io) plugin is installed,
the :py:class:`~pytest_notebook.nb_regression.NBRegressionFixture` will be initialised
with the settings and :py:class:`coverage.Coverage` object, that
`pytest-cov` has created.

If the `--nb-coverage` flag is set, then `nb_regression` will run coverage introspection,
and merge the data back into the main :py:class:`~coverage.Coverage` object.

In [27]:
%load_ext pytest_notebook.ipy_magic

In [43]:
%%pytest --disable-warnings --color=yes --cov=pytest_notebook --nb-coverage --log-cli-level=info

import logging
import importlib_resources
from pytest_notebook import example_nbs

def test_notebook(nb_regression):
    logging.getLogger(__name__).info(nb_regression)
    with importlib_resources.path(example_nbs, "coverage.ipynb") as path:
        nb_regression.check(str(path))

============================= test session starts ==============================
platform darwin -- Python 3.6.7, pytest-5.0.1, py-1.8.0, pluggy-0.12.0
rootdir: /private/var/folders/dm/b2qnkb_n3r72slmpxlfmcjvm00lbnd/T/tmpxcs1xsgm
plugins: cov-2.7.1, datadir-1.3.0, regressions-1.0.5, notebook-0.5.1
collected 1 item

test_ipycell.py::test_notebook 
-------------------------------- live log call ---------------------------------
INFO     test_ipycell:test_ipycell.py:7 NBRegressionFixture(exec_notebook=True, exec_cwd=None, exec_allow_errors=False, exec_timeout=120, coverage=True, cov_config='.coveragerc', cov_source=('pytest_notebook',), cov_merge=<coverage.control.Coverage object at 0x103ec3e48>, post_processors=('coalesce_streams',), process_resources={}, diff_replace=(), diff_ignore=('/cells/*/outputs/*/traceback',), diff_use_color=True, diff_color_words=False, force_regen=False)
INFO     pytest_notebook.execution:execution.py:147 About to execute notebook with 1 cells
INFO     pytest_n

This is also the case, when using the pytest file collection approach.

In [47]:
%%pytest --disable-warnings --color=yes --cov=pytest_notebook --log-cli-level=info

---
[pytest]
nb_coverage = True
nb_test_files = True
---

***
(dump_notebook(notebook), "test_notebook1.ipynb")
***

============================= test session starts ==============================
platform darwin -- Python 3.6.7, pytest-5.0.1, py-1.8.0, pluggy-0.12.0
rootdir: /private/var/folders/dm/b2qnkb_n3r72slmpxlfmcjvm00lbnd/T/tmpzc571yr2, inifile: pytest.ini
plugins: cov-2.7.1, datadir-1.3.0, regressions-1.0.5, notebook-0.5.1
collected 1 item

test_notebook1.ipynb::nbregression(test_notebook1) 
-------------------------------- live log call ---------------------------------
INFO     pytest_notebook.execution:execution.py:147 About to execute notebook with 1 cells
INFO     pytest_notebook.execution:execution.py:153 Executing notebook with kernel: python
INFO     pytest_notebook.execution:execution.py:155 Recording coverage for notebook
INFO     pytest_notebook.execution:execution.py:173 Executing cell 0
INFO     pytest_notebook.nb_regression:nb_regression.py:306 Merging coverage.
FAILED                                                                   [100%]

===================================